In [1]:
import mxnet as mx
from mxnet.gluon.model_zoo import vision
import os
import numpy as np
import glob
import pandas as pd
#from scipy.spatial.distance import cosine
from IPython.display import Image 

# set the context on CPU, switch to GPU if there is one available
ctx = mx.cpu()

import glob2
from tqdm import tqdm
import traceback

In [10]:
def prePrep(image_fn):
    '''
    inputs: full image fn
    output: numpy vector
    '''
    image = mx.image.imdecode(open(image_fn, 'rb').read()).astype(np.float32)
    resized = mx.image.resize_short(image, 224) #minimum 224x224 images
    cropped, crop_info = mx.image.center_crop(resized, (224, 224))
    normalized = mx.image.color_normalize(cropped/255,
                                          mean=mx.nd.array([0.485, 0.456, 0.406]),
                                          std=mx.nd.array([0.229, 0.224, 0.225])) 
    # the network expect batches of the form (N,3,224,224)
    flipped_axis = normalized.transpose((2,0,1))  # Flipping from (224, 224, 3) to (3, 224, 224)
    batchified = flipped_axis.expand_dims(axis=0) # change the shape from (3, 224, 224) to (1, 3, 224, 224)
    return densenet.features(batchified)[0].asnumpy()

In [11]:
%time densenet = vision.densenet201(pretrained=True)
type(densenet)


CPU times: user 563 ms, sys: 347 ms, total: 910 ms
Wall time: 791 ms


mxnet.gluon.model_zoo.vision.densenet.DenseNet

In [12]:
#all_header_files = glob2.glob('src/**/*_dogCrop.jpg')
fn_df_save = os.path.join(os.path.dirname(os.getcwd()), 'data', 'processed','0.0.6-whs-dogVectors.h5')
#fn_df_load = os.path.join(os.path.dirname(os.getcwd()), 'data', 'processed','0.0.3-whs-dogVectors.h5')
if not os.path.isdir(os.path.dirname(fn_df_save)):
    os.mkdir(os.path.dirname(fn_df_save))

df = pd.DataFrame()
df['fn'] = None
df['vector'] = None
fns = glob2.glob(os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw','**/*_dogCrop.jpg'))
df['fn'] = fns
'''for index, fn in enumerate(tqdm(fns)):
    #print('Processing: ',fn)
    if os.path.isdir(fn):
        fn = glob.iglob(os.path.join(fns, '*'))
    df.loc[index,'fn'] = fn
'''

print('Now Apply-ing')
%time df['vector'] = df['fn'].apply(lambda x: prePrep(x))
print('Done Apply-ing')
df.to_hdf(fn_df_save, key='df')
for i in range(3):
    print(df.loc[i])
df.tail()

Now Apply-ing
CPU times: user 1h 46min 26s, sys: 21min 7s, total: 2h 7min 34s
Wall time: 1h 18min 49s
Done Apply-ing
fn        /Users/officialbiznas/Documents/GitHub/ImageSi...
vector    [0.00018894675, 0.00061339705, 0.00079586904, ...
Name: 0, dtype: object
fn        /Users/officialbiznas/Documents/GitHub/ImageSi...
vector    [0.000119404496, 0.0006975044, 0.0014746733, 0...
Name: 1, dtype: object
fn        /Users/officialbiznas/Documents/GitHub/ImageSi...
vector    [0.00014220878, 0.0006950608, 0.00049114495, 0...
Name: 2, dtype: object


,fn,vector
13727,/Users/officialbiznas/Documents/GitHub/ImageSi...,"[3.1997246e-05, 0.0008480548, 0.0012421325, 0...."
13728,/Users/officialbiznas/Documents/GitHub/ImageSi...,"[8.132245e-05, 0.00022717057, 0.0008501709, 0...."
13729,/Users/officialbiznas/Documents/GitHub/ImageSi...,"[0.000101137535, 0.00038972704, 0.0012809561, ..."
13730,/Users/officialbiznas/Documents/GitHub/ImageSi...,"[0.00010224942, 0.0005458238, 0.0010494854, 0...."
13731,/Users/officialbiznas/Documents/GitHub/ImageSi...,"[4.8892372e-05, 0.00070208136, 0.001129705, 0...."


In [4]:
#df['vector'].loc[0]
#fn_save = os.path.join(os.path.dirname(os.getcwd()), 'data', 'processed','0.0.2-whs-dogVectors.pickle')
#df.to_pickle(fn_save)

In [5]:
'''
df = pd.read_pickle(fn_save)
labels = [df['fn'].loc[index].split('/')[-1] for index in range(df.count()[0])]
df_cross = pd.DataFrame(columns=labels, index=labels)
for index in range(df.count()[0]):
    for sub_index in range(df.count()[0]):
        u = df['vector'].loc[index]
        v = df['vector'].loc[sub_index]
        label1 = df['fn'].loc[index].split('/')[-1]
        label2 = df['fn'].loc[sub_index].split('/')[-1]
        df_cross[label1][label2] = np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v))
df_cross.tail()
'''

"\ndf = pd.read_pickle(fn_save)\nlabels = [df['fn'].loc[index].split('/')[-1] for index in range(df.count()[0])]\ndf_cross = pd.DataFrame(columns=labels, index=labels)\nfor index in range(df.count()[0]):\n    for sub_index in range(df.count()[0]):\n        u = df['vector'].loc[index]\n        v = df['vector'].loc[sub_index]\n        label1 = df['fn'].loc[index].split('/')[-1]\n        label2 = df['fn'].loc[sub_index].split('/')[-1]\n        df_cross[label1][label2] = np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v))\ndf_cross.tail()\n"